In [1]:
!pip install "uvicorn[standard]"
!pip install fastapi

     -------------------------------------- 58.3/58.3 kB 139.8 kB/s eta 0:00:00
     -------------------------------------- 58.3/58.3 kB 615.6 kB/s eta 0:00:00
     ------------------------------------ 143.0/143.0 kB 387.0 kB/s eta 0:00:00
     -------------------------------------- 270.9/270.9 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 57.0/57.0 kB 129.8 kB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 66.9/66.9 kB 1.8 MB/s eta 0:00:00


In [2]:
import h2o
from h2o.automl import H2OAutoML

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 7 mins
H2O_cluster_timezone:,Asia/Hong_Kong
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_y46ju_l41kbu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.575 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,32
H2O_cluster_status:,"locked, healthy"


In [4]:
saved_model = h2o.load_model("StackedEnsemble_BestOfFamily_1_AutoML_2_20230505_53230")

In [5]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class InputData(BaseModel):
    vol_moving_avg: float
    adj_close_rolling_med: float
    symbol: str
    etf: bool
    rsi: float
    tema: float

@app.post("/predict")
def predict_trading_volume(input_data: InputData):
    # Extract input values from the request body
    vol_moving_avg = input_data.vol_moving_avg
    adj_close_rolling_med = input_data.adj_close_rolling_med
    symbol = input_data.symbol
    etf = input_data.etf
    rsi = input_data.rsi
    tema = input_data.tema
    
    frame = h2o.H2OFrame({"Symbol":symbol, "etf":etf, "rsi":rsi, "tema":tema, "vol_moving_avg":vol_moving_avg, "adj_close_rolling_med":adj_close_rolling_med})
    
    # Return the predicted trading volume as a response
    return {"trading_volume": saved_model.predict(frame)}

In [6]:
import requests
